In [41]:
import pandas as pd
import numpy as np
from nltk.tag.hmm import HiddenMarkovModelTagger, HiddenMarkovModelTrainer
from utils import leer_csv, starts_ends_tokens, separar_spans_toxicos

In [2]:
train_data = leer_csv("../data/train_cleaned.csv")
train_data

,spans,text
0,"[8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 20, 21,...",another violent and aggressive immigrant killi...
1,"[33, 34, 35, 36, 37, 38, 39]","i am 56 years old, i am not your fucking junio..."
2,"[0, 1, 2, 3, 4]","damn, a whole family. sad indeed."
3,"[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]",what a knucklehead. how can anyone not know th...
4,"[32, 33, 34, 35, 36, 37, 38, 39, 40]","""who do you think should do the killing?""\n\na..."
...,...,...
7934,"[8, 9, 10, 11]",another fool pipes in.
7935,"[51, 52, 53, 54, 55, 56, 58, 59, 60, 61, 62, 6...",so if a restaurant owner puts up a sign saying...
7936,"[0, 1, 2, 4, 5, 6, 7, 8, 10, 11, 12, 13, 15, 1...",any faith that can't stand up to logic and rea...
7937,"[5, 6, 7, 8, 9, 10, 11, 12]",this idiotic. use the surplus to pay down the ...


# Obtener secuencias etiquetas

(palabra1,tag1), (palabra2,tag2),...

In [3]:
def etiquetar_secuencia(row):
    word_starts, word_ends = starts_ends_tokens(row.text)
    toxic_spans = separar_spans_toxicos(row)
    toxic_starts = [span[0] for span in toxic_spans]
    toxic_ends = [span[-1] for span in toxic_spans]
    
    secuencia = []
    
    for word_start, word_end in zip(word_starts, word_ends):
        tag = "T" if word_start in toxic_starts and word_end in toxic_ends else "N"
        secuencia.append(tag)
        
    secuencia = list(zip(row.text.split(), secuencia))
            
    return secuencia

In [4]:
etiquetar_secuencia(train_data.loc[0])

[('another', 'N'),
 ('violent', 'T'),
 ('and', 'T'),
 ('aggressive', 'T'),
 ('immigrant', 'T'),
 ('killing', 'N'),
 ('a', 'N'),
 ('innocent', 'N'),
 ('and', 'N'),
 ('intelligent', 'N'),
 ('us', 'N'),
 ('citizen....', 'N'),
 ('sarcasm', 'N')]

In [5]:
def get_vocab(df):
    tokens = df["text"].apply(str.split)
    vocab = set()
    for i,row in tokens.iteritems():
        vocab |= set(row)
    return vocab

vocab = get_vocab(train_data)

In [10]:
sentences = [etiquetar_secuencia(row) for row in train_data.itertuples()]

trainer = HiddenMarkovModelTrainer(states=["T", "N"], symbols=vocab)

model = trainer.train(sentences)

In [15]:
comment = train_data.loc[0,"text"].split()
comment

['another',
 'violent',
 'and',
 'aggressive',
 'immigrant',
 'killing',
 'a',
 'innocent',
 'and',
 'intelligent',
 'us',
 'citizen....',
 'sarcasm']

In [20]:
tagged = model.tag(comment)

In [43]:
def tags2offsets(tagged_comment):
    words = [word for word,tag in tagged_comment]
    tags = [tag for word, tag in tagged_comment]
    
    text = " ".join(words)
    starts, ends = starts_ends_tokens(text)
    offsets = []
    for i, (word, tag) in enumerate(tagged_comment):
        if tag == "T":
            offsets.extend(list( range(starts[i], ends[i]+1)) )
    return offsets

In [44]:
system_offsets = tags2offsets(tagged)

In [62]:
def f1_score(system_offsets , ground_truth):
    if ground_truth:
        system_offsets = set(system_offsets)
        ground_truth = set(ground_truth)

        interseccion = system_offsets | ground_truth
        precision = len(interseccion)/len(system_offsets) if system_offsets else 0
        recall = len(interseccion)/len(ground_truth) if ground_truth else 0
        f1 = (2*precision*recall)/(precision+recall)
        
    elif system_offsets:
        f1 = 0 # no hay verdaderas, pero hay en la predicción, se define como 0
    else:
        f1 = 1 
    return f1

In [60]:
tokenized_train_text = train_data.text.apply(str.split)
tagged_train_text = tokenized_train_text.apply(model.tag)

train_offsets = tagged_train_text.apply(tags2offsets)


In [63]:
train_scores = [f1_score(system_offsets, ground_truth) 
                for system_offsets, ground_truth in zip(train_offsets, train_data.spans)]
np.mean(train_scores)

0.8202437086888265

Ahora con trial

In [65]:
trial_data = leer_csv("../data/trial_cleaned.csv")

In [66]:
tokenized_test_test = trial_data.text.apply(str.split)
tagged_test_text = tokenized_test_test.apply(model.tag)

test_offsets = tagged_test_text.apply(tags2offsets)

test_scores = [f1_score(system_offsets, ground_truth) 
                for system_offsets, ground_truth in zip(test_offsets, trial_data.spans)]
np.mean(test_scores)

1.4416600298577078

Error en la implementación de F1?